# Predicting with DetecTreeRGB

This script predicts on new tiles with a trained model.

In [ ]:
### install dependencies:

# might need to install opencv
#!pip3 install Cython cupy-cuda112 cupy-cuda102   # Probably only need one of these but it works so I'm gonna leave it for now.
!pip -q install pyyaml==5.1

# Torch 1.8.1 does not work despite being the latest stable release. As such, use 1.7. 
# Cuda version on MAGEOHub is currently 11.3 - cu110 all works fine.

# To be tried once the bug with torch 1.8 has been fixed 
#!pip3 install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

#!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
#!python -m pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html

### colab has updated it's pytorch version...need new Detectron2 version
!pip -q install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

     |████████████████████████████████| 274 kB 6.3 MB/s 
     |████████████████████████████████| 7.0 MB 665 kB/s 
     |████████████████████████████████| 49 kB 2.8 MB/s 
     |████████████████████████████████| 145 kB 8.7 MB/s 
     |████████████████████████████████| 74 kB 3.7 MB/s 
     |████████████████████████████████| 130 kB 53.5 MB/s 
     |████████████████████████████████| 748 kB 76.3 MB/s 
     |████████████████████████████████| 843 kB 82.9 MB/s 
     |████████████████████████████████| 112 kB 63.3 MB/s 


In [ ]:
# Import torch and check versions and cuda availability, and resources
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
!nvcc --version
!nvidia-smi

from IPython.display import display, clear_output

1.10.0+cu111 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Tue Feb 22 09:08:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|========================

In [ ]:
# Some basic imports:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# tensorboard?
#%load_ext tensorboard
#%tensorboard --logdir output

# necessary libraries
import pandas as pd
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import json
import glob

#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode


In [ ]:
# Make sure the inevitable error messages are useful.
CUDA_LAUNCH_BLOCKING="1"

In [ ]:
# Setup to predict on new images, here setting up for the trees_test dataset, but can also use this setup
# for predicting on individual images as seen 2 cells down

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# if we want to make predictions using a CPU
cfg.MODEL.DEVICE='cuda'

### path to the saved pre-trained model weights
cfg.MODEL.WEIGHTS = '/content/drive/Shareddrives/detectreeRGB/weights/model_final.pth'

# set confidence threshold at which we predict
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.15

predictor = DefaultPredictor(cfg)

In [ ]:
### Ok this now works...

# loop through all pngs, once they have been tiled with tiling script

for filepath in glob.iglob('/content/drive/Shareddrives/detectreeRGB/benchmark/P15_tiled_140222/pngs/*.png'):
    print(filepath)
    # first, let's extract the x and y origins (remember x_origin is on the left, y_origin is the top!)
    path, filename = os.path.split(filepath)
    #print(filename)
    root, ext = os.path.splitext(filename)
    #print(root)
    tile_standard, x_origin, y_origin = root.rsplit("_", 2)
    print('x origin:', x_origin)
    print('y origin:', y_origin)    
    
    # now have a look at the image shape
    img = cv2.imread(filepath)
    print('png shape:', img.shape)
    outputs = predictor(img)
    mask_array = outputs['instances'].pred_masks.cpu().numpy()

    # get confidence scores too 
    mask_array_scores = outputs['instances'].scores.cpu().numpy()

    num_instances = mask_array.shape[0]
    mask_array_instance = []
    output = np.zeros_like(mask_array) 

    mask_array_instance.append(mask_array)
    output = np.where(mask_array_instance[0] == True, 255, output)
    fresh_output = output.astype(float) #np.float has been replaced here to suppress warning
    # are these right?
    x_scaling = (img.shape[1] * resolution)/fresh_output.shape[2]
    #x_scaling = 10
    y_scaling = (img.shape[0] * resolution)/fresh_output.shape[1]
    #y_scaling = 10
    # this is an affine transform. This needs to be altered significantly.
    # it does! This needs to be fixed nicely.
    # removed buffer -20, +120
    # So actually, this transform is going from the wrong origin...it is going from the round number...but it shouldn't...
    # But then the y origin is very accurate...
    # then also we need to sort out this thing about transforming from the top! But from the top works...because it is exact...
    # think y-axis is going to need changing
    # these +20s and -20s are wrong...and the scaling is wrong by a bit
    # this buffer is wrong FOR the ones on the x and y origins! No buffer wanted for those!
    # need something like if filepath = origin_x: do not add buffer, if filepath =/= origin_x, do add buffer
    if x_origin == str(data.bounds[0]) and y_origin == str(data.bounds[3]):
      transform = from_origin(float(x_origin), float(y_origin), x_scaling, y_scaling)
    elif x_origin == str(data.bounds[0]):
      transform = from_origin(float(x_origin), float(y_origin)+20, x_scaling, y_scaling)
    elif y_origin == str(data.bounds[3]):
      transform = from_origin(float(x_origin)-20, float(y_origin), x_scaling, y_scaling)
    else:
      transform = from_origin(float(x_origin)-20, float(y_origin)+20, x_scaling, y_scaling)



    output_raster =  '/content/drive/Shareddrives/detectreeRGB/benchmark/P15_tiled_140222/rasters/' + 'predicted_raster_' + root + '.tif'

    ### perhaps here try to remove the new_dataset.write() function...can we do it without having to save this raster?
    new_dataset = rasterio.open(output_raster, 'w', driver='GTiff',
                                    height = fresh_output.shape[1], width = fresh_output.shape[2], count = fresh_output.shape[0],
                                    dtype=str(fresh_output.dtype),
                                    crs='EPSG:4326',  
                                    transform=transform)

    new_dataset.write(fresh_output)
    new_dataset.close()

    with rasterio.open(output_raster) as src:
        shp_schema = {'geometry': 'MultiPolygon','properties': {'pixelvalue': 'int', 'score': 'float'}}    

        crs = src.crs
        for i in range(src.count):
            src_band = src.read(i+1)
            src_band = np.float32(src_band)
            
            # this is odd, perhaps should be i - 1
            conf = mask_array_scores[i-2]
            # Keep track of unique pixel values in the input band...I should probably hash this out
            unique_values = np.unique(src_band)
            # Polygonize with Rasterio. `shapes()` returns an iterable
            # of (geom, value) as tuples
            shapes = list(rasterio.features.shapes(src_band, transform=src.transform))

          
            if i == 0:
                with fiona.open('/content/drive/Shareddrives/detectreeRGB/benchmark/P15_tiled_140222/shapes/' + 'predicted_polygons_' + root + '.shp', 'w', 'ESRI Shapefile',
                                shp_schema, crs) as shp:
                    polygons = [shape(geom) for geom, value in shapes if value == 255.0]                                        
                    multipolygon = MultiPolygon(polygons)
                            # simplify not needed here
                            #multipolygon = multipolygon_a.simplify(0.1, preserve_topology=False)                    
                    shp.write({
                              'geometry': mapping(multipolygon),
                              'properties': {'pixelvalue': int(unique_values[1]), 'score': float(conf)} 
                              })
            else:
                with fiona.open('/content/drive/Shareddrives/detectreeRGB/benchmark/P15_tiled_140222/shapes/' + 'predicted_polygons_' + root + '.shp', 'a', 'ESRI Shapefile',
                                shp_schema, crs) as shp:
                    polygons = [shape(geom) for geom, value in shapes if value == 255.0]                                        
                    multipolygon = MultiPolygon(polygons)
                            # simplify not needed here
                            #multipolygon = multipolygon_a.simplify(0.1, preserve_topology=False)                    
                    shp.write({
                              'geometry': mapping(multipolygon),
                              'properties': {'pixelvalue': int(unique_values[1]), 'score': float(conf)} 
                              })
        


/content/drive/Shareddrives/detectreeRGB/benchmark/P15_tiled_140222/pngs/tile_286241.82599999994_583833.138.png
x origin: 286241.82599999994
y origin: 583833.138
png shape: (1200, 1200, 3)


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


/content/drive/Shareddrives/detectreeRGB/benchmark/P15_tiled_140222/pngs/tile_286241.82599999994_583733.138.png
x origin: 286241.82599999994
y origin: 583733.138
png shape: (1400, 1200, 3)
/content/drive/Shareddrives/detectreeRGB/benchmark/P15_tiled_140222/pngs/tile_286241.82599999994_583633.138.png
x origin: 286241.82599999994
y origin: 583633.138
png shape: (1401, 1200, 3)
/content/drive/Shareddrives/detectreeRGB/benchmark/P15_tiled_140222/pngs/tile_286241.82599999994_583533.138.png
x origin: 286241.82599999994
y origin: 583533.138
png shape: (671, 1200, 3)
/content/drive/Shareddrives/detectreeRGB/benchmark/P15_tiled_140222/pngs/tile_286341.82599999994_583833.138.png
x origin: 286341.82599999994
y origin: 583833.138
png shape: (1200, 1400, 3)
/content/drive/Shareddrives/detectreeRGB/benchmark/P15_tiled_140222/pngs/tile_286341.82599999994_583733.138.png
x origin: 286341.82599999994
y origin: 583733.138
png shape: (1400, 1400, 3)
/content/drive/Shareddrives/detectreeRGB/benchmark/P15_t